### This file takes the files that Julia creates as well as the number of mg normalized cells per ml and calculates the number of transcripts per cell for every gene, GEN, date

rerun on 2021-12-07 after correcting for salinity of media samples

In [27]:
import pandas as pd
from pandas import DataFrame
import os
import sys
import csv
from pathlib import Path
import seaborn as sns
sns.set()
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.show()
from collections import defaultdict
import os.path as op
import glob

os.chdir('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/GORG_recruitment/RNA_summaries/')
outdir='/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/GORG_recruitment/RNA_summaries/'

In [28]:
mg_abundance=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/GoM_Metagenome_analysis/GoM_metagenome_calculated_Genus_cells_per_ml.csv')
mg_abundance

,genus,20171102_genome_size_normalized_metagenome_abundance,20181030_genome_size_normalized_metagenome_abundance,20190402_genome_size_normalized_metagenome_abundance,20190709_genome_size_normalized_metagenome_abundance,20171102_genome_size_normalized_metagenome_cells/ml,20181030_genome_size_normalized_metagenome_cells/ml,20190402_genome_size_normalized_metagenome_cells/ml,20190709_genome_size_normalized_metagenome_cells/ml,GTDB_genus,full_GTDBtk_classification,#_of_cells,genus_size_rank
0,AAA164-E04,0.000543,0.001322,0.000225,0.000294,842.193966,1916.455808,369.342772,857.676906,AAA164-E04,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,37.0,25.0
1,AAA536-G10,0.011273,0.011666,0.008782,0.006069,17472.787073,16915.541760,14403.280557,17721.566151,AAA536-G10,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,48.0,17.0
2,AG-337-I02,0.001437,0.006798,0.000656,0.000343,2227.335509,9857.221878,1075.339238,1002.756455,AG-337-I02,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,14.0,51.0
3,AG-414-E02,0.000081,0.000303,0.000245,0.000044,125.191537,438.765267,401.157950,128.286002,AG-414-E02,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,1.0,176.0
4,AG-422-B15,0.001213,0.002285,0.008358,0.002307,1880.679178,3313.158567,13707.625915,6736.371699,AG-422-B15,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,14.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,Verrucomicrobiales,0.000172,0.000274,0.000033,0.000096,265.899026,397.482106,54.179686,280.890811,Verrucomicrobiales,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,5.0,107.0
164,Vibrio,0.000401,0.000428,0.000325,0.000330,622.265686,620.661123,533.774653,964.925624,Vibrio,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,1.0,301.0
165,Vicingaceae,0.000505,0.000324,0.004361,0.001741,782.480532,470.155212,7151.270318,5084.185705,Vicingaceae,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,3.0,138.0
166,Winogradskyella,0.002990,0.001380,0.006467,0.003672,4635.264065,2001.192608,10605.994111,10722.976925,Winogradskyella,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,9.0,77.0


In [29]:
#Read number to transcript conversion factors 
#These numbers came from a file on my computer and were hard calculated by me
# using the MTST number of reads per MTST standard 
# Users/jmunson-mcgee/Desktop/RNA_misc/MTST_standard_calculations.xlsx 
#Multiply the number of reads mapped to  a transcript in a library this value to get the number of transcripts per ml. 
# I hae already taken into account the sample size of each sample

#Conversion factors
conversion_171102=210.7600209
conversion_181030=67.53617178
conversion_190402=103.6150407
conversion_190709=179.1259172


In [30]:
df_171102=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/GORG_recruitment/summaries/All_171102_GoM_RNA_seq_bbmerge_reads_annotated_read_count_by_genus_and_prokka_annotations.csv')

df_171102['20171102_transcripts_per_ml']=df_171102['exclusive']*conversion_171102

mg_abundance20171102=mg_abundance[['genus', '20171102_genome_size_normalized_metagenome_cells/ml']]

df_171102=df_171102.merge(mg_abundance20171102, how='left', on='genus')
df_171102['20171102_transcripts_per_cell']=df_171102['20171102_transcripts_per_ml']/df_171102['20171102_genome_size_normalized_metagenome_cells/ml']
df_171102.to_csv(outdir+'All_171102_GoM_RNA_seq_bbmerge_reads_annotated_reads_by_gen_and_prokka_annotation_transcripts_per_cell.csv')

df_171102

,genus,prokka_function,prokka_gene,ec_number,exclusive,shared,20171102_transcripts_per_ml,20171102_genome_size_normalized_metagenome_cells/ml,20171102_transcripts_per_cell
0,Unclassified,(2E%2C6E)-farnesyl_diphosphate_synthase,.,2.5.1.10,0,5,0.000000,230950.552049,0.000000
1,Unclassified,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)),comC,1.1.1.338,0,1,0.000000,230950.552049,0.000000
2,Unclassified,(2R)-sulfolactate_sulfo-lyase_subunit_alpha,suyA,4.4.1.24,0,7,0.000000,230950.552049,0.000000
3,Unclassified,(2R)-sulfolactate_sulfo-lyase_subunit_beta,suyB,4.4.1.24,0,37,0.000000,230950.552049,0.000000
4,Unclassified,(2S)-methylsuccinyl-CoA_dehydrogenase,mcd,1.3.8.12,0,14,0.000000,230950.552049,0.000000
...,...,...,...,...,...,...,...,...,...
106615,Arenitalea,putative_peptidyl-prolyl_cis-trans_isomerase,ppiB_2,5.2.1.8,7,0,1475.320146,47.855594,30.828583
106616,Arenitalea,putative_peroxiredoxin,tsaA,1.11.1.15,0,2,0.000000,47.855594,0.000000
106617,Arenitalea,putative_protein,.,.,0,1,0.000000,47.855594,0.000000
106618,Arenitalea,tRNA1(Val)_(adenine(37)-N6)-methyltransferase,yfiC,2.1.1.223,0,1,0.000000,47.855594,0.000000


In [31]:
df_181030=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/GORG_recruitment/summaries/All_181030_GoM_RNA_seq_bbmerge_reads_annotated_read_count_by_genus_and_prokka_annotations.csv')

df_181030['20181030_transcripts_per_ml']=df_181030['exclusive']*conversion_181030

mg_abundance20181030=mg_abundance[['genus', '20181030_genome_size_normalized_metagenome_cells/ml']]

df_181030=df_181030.merge(mg_abundance20181030, how='left', on='genus')
df_181030['20181030_transcripts_per_cell']=df_181030['20181030_transcripts_per_ml']/df_181030['20181030_genome_size_normalized_metagenome_cells/ml']
df_181030.to_csv(outdir+'All_181030_GoM_RNA_seq_bbmerge_reads_annotated_reads_by_gen_and_prokka_annotation_transcripts_per_cell.csv')

df_181030

,genus,prokka_function,prokka_gene,ec_number,exclusive,shared,20181030_transcripts_per_ml,20181030_genome_size_normalized_metagenome_cells/ml,20181030_transcripts_per_cell
0,Unclassified,(2E%2C6E)-farnesyl_diphosphate_synthase,.,2.5.1.10,0,4,0.000000,194593.423578,0.000000
1,Unclassified,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)),comC,1.1.1.338,0,3,0.000000,194593.423578,0.000000
2,Unclassified,(2R)-sulfolactate_sulfo-lyase_subunit_alpha,suyA,4.4.1.24,0,6,0.000000,194593.423578,0.000000
3,Unclassified,(2R)-sulfolactate_sulfo-lyase_subunit_beta,suyB,4.4.1.24,0,70,0.000000,194593.423578,0.000000
4,Unclassified,(2S)-methylsuccinyl-CoA_dehydrogenase,mcd,1.3.8.12,0,17,0.000000,194593.423578,0.000000
...,...,...,...,...,...,...,...,...,...
79240,UBA5081,hypothetical_protein,.,.,24,27,1620.868123,28.974125,55.941917
79241,UBA5081,putative_ABC_transporter_ATP-binding_protein_YbiT,ybiT,.,0,1,0.000000,28.974125,0.000000
79242,UBA5081,putative_protein,.,.,0,3,0.000000,28.974125,0.000000
79243,UBA5081,putative_zinc_protease,.,3.4.24.-,0,1,0.000000,28.974125,0.000000


In [32]:
df_190402=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/GORG_recruitment/summaries/All_190402_GoM_RNA_seq_bbmerge_reads_annotated_read_count_by_genus_and_prokka_annotations.csv')

df_190402['20190402_transcripts_per_ml']=df_190402['exclusive']*conversion_190402

mg_abundance20190402=mg_abundance[['genus', '20190402_genome_size_normalized_metagenome_cells/ml']]

df_190402=df_190402.merge(mg_abundance20190402, how='left', on='genus')
df_190402['20190402_transcripts_per_cell']=df_190402['20190402_transcripts_per_ml']/df_190402['20190402_genome_size_normalized_metagenome_cells/ml']
df_190402.to_csv(outdir+'All_190402_GoM_RNA_seq_bbmerge_reads_annotated_reads_by_gen_and_prokka_annotation_transcripts_per_cell.csv')

df_190402

,genus,prokka_function,prokka_gene,ec_number,exclusive,shared,20190402_transcripts_per_ml,20190402_genome_size_normalized_metagenome_cells/ml,20190402_transcripts_per_cell
0,MB11C04,1%2C5-anhydro-D-fructose_reductase,afr,1.1.1.292,4,4,414.460163,1613.425727,0.256882
1,MB11C04,10_kDa_chaperonin,groS,.,8,0,828.920326,1613.425727,0.513764
2,MB11C04,10_kDa_chaperonin,groS_1,.,0,1,0.000000,1613.425727,0.000000
3,MB11C04,2%2C3-bisphosphoglycerate-dependent_phosphogly...,gpmA,5.4.2.11,52,3,5387.982116,1613.425727,3.339467
4,MB11C04,2-dehydro-3-deoxygluconokinase,kdgK,2.7.1.45,1,0,103.615041,1613.425727,0.064221
...,...,...,...,...,...,...,...,...,...
93957,UBA7446,Valine--tRNA_ligase,valS,6.1.1.9,0,1,0.000000,258.132811,0.000000
93958,UBA7446,hypothetical_protein,.,.,13,24,1346.995529,258.132811,5.218227
93959,UBA7446,putative_peptidyl-prolyl_cis-trans_isomerase,ppiB,5.2.1.8,0,1,0.000000,258.132811,0.000000
93960,UBA7446,putative_peptidyl-prolyl_cis-trans_isomerase,ppiB_1,5.2.1.8,7,1,725.305285,258.132811,2.809814


In [33]:
df_190709=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/GORG_recruitment/summaries/All_190709_GoM_RNA_seq_bbmerge_reads_annotated_read_count_by_genus_and_prokka_annotations.csv')

df_190709['20190709_transcripts_per_ml']=df_190709['exclusive']*conversion_190709

mg_abundance20190709=mg_abundance[['genus', '20190709_genome_size_normalized_metagenome_cells/ml']]

df_190709=df_190709.merge(mg_abundance20190709, how='left', on='genus')
df_190709['20190709_transcripts_per_cell']=df_190709['20190709_transcripts_per_ml']/df_190709['20190709_genome_size_normalized_metagenome_cells/ml']
df_190709.to_csv(outdir+'All_190709_GoM_RNA_seq_bbmerge_reads_annotated_reads_by_gen_and_prokka_annotation_transcripts_per_cell.csv')

df_190709

,genus,prokka_function,prokka_gene,ec_number,exclusive,shared,20190709_transcripts_per_ml,20190709_genome_size_normalized_metagenome_cells/ml,20190709_transcripts_per_cell
0,SHLJ01,(2E%2C6E)-farnesyl_diphosphate_synthase,.,2.5.1.10,5,0,895.629586,2163.168462,0.414036
1,SHLJ01,1-pyrroline-5-carboxylate_dehydrogenase_2,putC,1.2.1.88,0,2,0.000000,2163.168462,0.000000
2,SHLJ01,10_kDa_chaperonin_1,groS1,.,0,6,0.000000,2163.168462,0.000000
3,SHLJ01,10_kDa_chaperonin_3,groS3,.,1,15,179.125917,2163.168462,0.082807
4,SHLJ01,2-oxoglutarate_dehydrogenase_E1_component,sucA,1.2.4.2,9,0,1612.133255,2163.168462,0.745265
...,...,...,...,...,...,...,...,...,...
76904,UBA1611,Valine--tRNA_ligase,valS,6.1.1.9,1,1,179.125917,86.696810,2.066119
76905,UBA1611,dTDP-glucose_4%2C6-dehydratase_2,rffG,4.2.1.46,0,1,0.000000,86.696810,0.000000
76906,UBA1611,hypothetical_protein,.,.,12,30,2149.511006,86.696810,24.793427
76907,UBA1611,putative_3-hydroxyacyl-CoA_dehydrogenase,fadN,1.1.1.35,4,0,716.503669,86.696810,8.264476


In [34]:

transcripts_20171102=pd.read_csv(outdir+'All_171102_GoM_RNA_seq_bbmerge_reads_annotated_reads_by_gen_and_prokka_annotation_transcripts_per_cell.csv', 
                                usecols=['genus', 'prokka_function', 'prokka_gene', 'ec_number', '20171102_transcripts_per_cell'])
transcripts_20181030=pd.read_csv(outdir+'All_181030_GoM_RNA_seq_bbmerge_reads_annotated_reads_by_gen_and_prokka_annotation_transcripts_per_cell.csv', 
                                usecols=['genus', 'prokka_function', 'prokka_gene', 'ec_number', '20181030_transcripts_per_cell'])
transcripts_20190402=pd.read_csv(outdir+'All_190402_GoM_RNA_seq_bbmerge_reads_annotated_reads_by_gen_and_prokka_annotation_transcripts_per_cell.csv', 
                                usecols=['genus', 'prokka_function', 'prokka_gene', 'ec_number', '20190402_transcripts_per_cell'])
transcripts_20190709=pd.read_csv(outdir+'All_190709_GoM_RNA_seq_bbmerge_reads_annotated_reads_by_gen_and_prokka_annotation_transcripts_per_cell.csv', 
                                usecols=['genus', 'prokka_function', 'prokka_gene', 'ec_number', '20190709_transcripts_per_cell'])


transcripts_master=transcripts_20171102.merge(transcripts_20181030, how='outer', 
                                            on=['genus', 'prokka_function', 'prokka_gene', 'ec_number'])
transcripts_master=transcripts_master.merge(transcripts_20190402, how='outer', 
                                            on=['genus', 'prokka_function', 'prokka_gene', 'ec_number'])
transcripts_master=transcripts_master.merge(transcripts_20190709, how='outer', 
                                            on=['genus', 'prokka_function', 'prokka_gene', 'ec_number'])
transcripts_master.to_csv(outdir+'All_GoM_RNA_seq_transcripts_per_cell_by_genus_and_Prokka_annotations.csv')

transcripts_master

,genus,prokka_function,prokka_gene,ec_number,20171102_transcripts_per_cell,20181030_transcripts_per_cell,20190402_transcripts_per_cell,20190709_transcripts_per_cell
0,Unclassified,(2E%2C6E)-farnesyl_diphosphate_synthase,.,2.5.1.10,0.0,0.0,0.0,0.0
1,Unclassified,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)),comC,1.1.1.338,0.0,0.0,0.0,0.0
2,Unclassified,(2R)-sulfolactate_sulfo-lyase_subunit_alpha,suyA,4.4.1.24,0.0,0.0,0.0,0.0
3,Unclassified,(2R)-sulfolactate_sulfo-lyase_subunit_beta,suyB,4.4.1.24,0.0,0.0,0.0,0.0
4,Unclassified,(2S)-methylsuccinyl-CoA_dehydrogenase,mcd,1.3.8.12,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
167249,UBA1611,Chondroitin_synthase,kfoC,.,NaN,NaN,NaN,0.0
167250,UBA1611,DNA_gyrase_subunit_B,gyrB_1,5.99.1.3,NaN,NaN,NaN,0.0
167251,UBA1611,Dipeptidyl_aminopeptidase_BI,dapb1,3.4.14.-,NaN,NaN,NaN,0.0
167252,UBA1611,Fe-S_protein_maturation_auxiliary_factor_SufT,sufT,.,NaN,NaN,NaN,0.0


In [35]:
transcripts_20171102['date']=20171102
transcripts_20171102=transcripts_20171102.rename(columns={'20171102_transcripts_per_cell':'transcripts_per_cell'})

transcripts_20181030['date']=20181030
transcripts_20181030=transcripts_20181030.rename(columns={'20181030_transcripts_per_cell':'transcripts_per_cell'})
transcripts_20190402['date']=20190402
transcripts_20190402=transcripts_20190402.rename(columns={'20190402_transcripts_per_cell':'transcripts_per_cell'})
transcripts_20190709['date']=20190709
transcripts_20190709=transcripts_20190709.rename(columns={'20190709_transcripts_per_cell':'transcripts_per_cell'})

transcripts_20171102

,genus,prokka_function,prokka_gene,ec_number,transcripts_per_cell,date
0,Unclassified,(2E%2C6E)-farnesyl_diphosphate_synthase,.,2.5.1.10,0.000000,20171102
1,Unclassified,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)),comC,1.1.1.338,0.000000,20171102
2,Unclassified,(2R)-sulfolactate_sulfo-lyase_subunit_alpha,suyA,4.4.1.24,0.000000,20171102
3,Unclassified,(2R)-sulfolactate_sulfo-lyase_subunit_beta,suyB,4.4.1.24,0.000000,20171102
4,Unclassified,(2S)-methylsuccinyl-CoA_dehydrogenase,mcd,1.3.8.12,0.000000,20171102
...,...,...,...,...,...,...
106615,Arenitalea,putative_peptidyl-prolyl_cis-trans_isomerase,ppiB_2,5.2.1.8,30.828583,20171102
106616,Arenitalea,putative_peroxiredoxin,tsaA,1.11.1.15,0.000000,20171102
106617,Arenitalea,putative_protein,.,.,0.000000,20171102
106618,Arenitalea,tRNA1(Val)_(adenine(37)-N6)-methyltransferase,yfiC,2.1.1.223,0.000000,20171102


In [36]:
transcripts_stack=transcripts_20171102.append(transcripts_20181030, ignore_index=True)

transcripts_stack=transcripts_stack.append(transcripts_20190402, ignore_index=True)
transcripts_stack=transcripts_stack.append(transcripts_20190709, ignore_index=True)
transcripts_stack=transcripts_stack[transcripts_stack['transcripts_per_cell'].notna()]

transcripts_stack.to_csv(outdir+'All_GoM_RNA_seq_transcripts_per_cell_by_genus_and_prokka_anotation_stacked.csv')
transcripts_stack

,genus,prokka_function,prokka_gene,ec_number,transcripts_per_cell,date
0,Unclassified,(2E%2C6E)-farnesyl_diphosphate_synthase,.,2.5.1.10,0.000000,20171102
1,Unclassified,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)),comC,1.1.1.338,0.000000,20171102
2,Unclassified,(2R)-sulfolactate_sulfo-lyase_subunit_alpha,suyA,4.4.1.24,0.000000,20171102
3,Unclassified,(2R)-sulfolactate_sulfo-lyase_subunit_beta,suyB,4.4.1.24,0.000000,20171102
4,Unclassified,(2S)-methylsuccinyl-CoA_dehydrogenase,mcd,1.3.8.12,0.000000,20171102
...,...,...,...,...,...,...
356731,UBA1611,Valine--tRNA_ligase,valS,6.1.1.9,2.066119,20190709
356732,UBA1611,dTDP-glucose_4%2C6-dehydratase_2,rffG,4.2.1.46,0.000000,20190709
356733,UBA1611,hypothetical_protein,.,.,24.793427,20190709
356734,UBA1611,putative_3-hydroxyacyl-CoA_dehydrogenase,fadN,1.1.1.35,8.264476,20190709


In [37]:
transcripts_stack=transcripts_stack.drop_duplicates()
transcripts_stack['prokka_annotation']=(transcripts_stack['prokka_function']+ ":"+transcripts_stack['prokka_gene']+":"+
                                        transcripts_stack['ec_number'])

transcripts_stack
transcripts_stack=transcripts_stack.drop_duplicates()
prokka_pivot=transcripts_stack.pivot(index=['genus','date'], values='transcripts_per_cell', columns='prokka_annotation')
prokka_pivot.to_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/GORG_recruitment/RNA_summaries/All_GoM_RNA_seq_transcripts_per_cell_by_genus_and_prokka_annotation_pivot.csv')

master_pivot=prokka_pivot.reset_index()
master_pivot

/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


prokka_annotation,genus,date,(-)-trans-carveol_dehydrogenase:limC:1.1.1.-,(2E%2C6E)-farnesyl_diphosphate_synthase:.:2.5.1.10,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_1:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_2:1.1.1.338,(2R)-sulfolactate_sulfo-lyase_subunit_alpha:suyA:4.4.1.24,(2R)-sulfolactate_sulfo-lyase_subunit_beta:suyB:4.4.1.24,(2S)-methylsuccinyl-CoA_dehydrogenase:mcd:1.3.8.12,...,tRNA_threonylcarbamoyladenosine_biosynthesis_protein_TsaE:tsaE:.,tRNA_threonylcarbamoyladenosine_biosynthesis_protein_TsaE:tsaE_1:.,tRNA_threonylcarbamoyladenosine_dehydratase:tcdA:6.1.-.-,tRNA_uridine_5-carboxymethylaminomethyl_modification_enzyme_MnmG:mnmG:.,tRNA_uridine_5-carboxymethylaminomethyl_modification_enzyme_MnmG:mnmG_1:.,tRNA_uridine_5-carboxymethylaminomethyl_modification_enzyme_MnmG:mnmG_2:.,tRNA_uridine_5-carboxymethylaminomethyl_modification_enzyme_MnmG:mnmG_3:.,zeta-carotene-forming_phytoene_desaturase:carA2:1.3.99.29,zeta-carotene-forming_phytoene_desaturase:carA2_1:1.3.99.29,zeta-carotene-forming_phytoene_desaturase:carA2_2:1.3.99.29
0,AAA164-E04,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.250251,NaN,NaN,NaN,NaN,NaN,NaN
1,AAA164-E04,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAA164-E04,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAA164-E04,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.044251,NaN,NaN,NaN,NaN,NaN,NaN
4,AAA536-G10,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.108560,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,Winogradskyella,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.03341,NaN,0.066819,NaN,NaN,NaN,NaN,NaN,NaN
656,Yoonia,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.968799,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
657,Yoonia,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
658,Yoonia,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032036,...,0.008009,NaN,NaN,0.024027,NaN,NaN,NaN,NaN,NaN,NaN


# Next Steps
I need to merge this file with the O2 consumption file as well as the geochemical data. Then once that is done I need to remove the GENs that I do not want to include. I have been removing GENs with <10 SAGs in an attempt to try and ensure that I have at least decent coverage of the genome for a specific genus.


In [38]:

transcript_data=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/O2_consumption/All_GoM_RNA_seq_transcripts_per_cell_by_genus_and_function_stacked_weighted_avg_O2_consumed_ALL_genes_all_Genus.csv',
                            index_col=0)

transcript_data=transcript_data.drop_duplicates()

transcript_pivot=transcript_data.pivot(index=['genus','date'], values='transcripts_per_cell', columns='ko')
transcript_pivot.to_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/GORG_recruitment/RNA_summaries/All_GoM_RNA_seq_transcripts_per_cell_by_genus_and_KO.csv')


O2_data=transcript_data[['genus', 'date', 'Weighted_avg_O2_consumed_per_cell']]
O2_data=O2_data.drop_duplicates()

O2_data
#transcript_data

,genus,date,Weighted_avg_O2_consumed_per_cell
0,Unclassified,20171102,23.337911
2643,LFER01,20171102,0.002330
3987,Akkermansiaceae,20171102,0.002330
4497,SHLJ01,20171102,0.002330
5015,UBA4582,20171102,0.002330
...,...,...,...
254987,GCA-002724395,20190709,0.001999
255115,Nitrosopelagicus,20190709,0.001999
255158,Sulfitobacter,20190709,0.001999
255312,UBA1096,20190709,0.001999


In [39]:
master_pivot=master_pivot.merge(O2_data, on=['genus', 'date'])

master_pivot

,genus,date,(-)-trans-carveol_dehydrogenase:limC:1.1.1.-,(2E%2C6E)-farnesyl_diphosphate_synthase:.:2.5.1.10,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_1:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_2:1.1.1.338,(2R)-sulfolactate_sulfo-lyase_subunit_alpha:suyA:4.4.1.24,(2R)-sulfolactate_sulfo-lyase_subunit_beta:suyB:4.4.1.24,(2S)-methylsuccinyl-CoA_dehydrogenase:mcd:1.3.8.12,...,tRNA_threonylcarbamoyladenosine_biosynthesis_protein_TsaE:tsaE_1:.,tRNA_threonylcarbamoyladenosine_dehydratase:tcdA:6.1.-.-,tRNA_uridine_5-carboxymethylaminomethyl_modification_enzyme_MnmG:mnmG:.,tRNA_uridine_5-carboxymethylaminomethyl_modification_enzyme_MnmG:mnmG_1:.,tRNA_uridine_5-carboxymethylaminomethyl_modification_enzyme_MnmG:mnmG_2:.,tRNA_uridine_5-carboxymethylaminomethyl_modification_enzyme_MnmG:mnmG_3:.,zeta-carotene-forming_phytoene_desaturase:carA2:1.3.99.29,zeta-carotene-forming_phytoene_desaturase:carA2_1:1.3.99.29,zeta-carotene-forming_phytoene_desaturase:carA2_2:1.3.99.29,Weighted_avg_O2_consumed_per_cell
0,AAA164-E04,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.250251,NaN,NaN,NaN,NaN,NaN,NaN,0.002330
1,AAA164-E04,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002050
2,AAA164-E04,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002101
3,AAA164-E04,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.044251,NaN,NaN,NaN,NaN,NaN,NaN,0.001999
4,AAA536-G10,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.108560,NaN,NaN,NaN,NaN,NaN,NaN,0.002330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,Winogradskyella,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.03341,NaN,0.066819,NaN,NaN,NaN,NaN,NaN,NaN,0.001999
656,Yoonia,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.968799,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002330
657,Yoonia,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002050
658,Yoonia,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032036,...,NaN,NaN,0.024027,NaN,NaN,NaN,NaN,NaN,NaN,0.002101


In [40]:
GEN_data=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/Genus_statistics.csv', index_col=0)

GEN_data

,GTDB_genus,full_GTDBtk_classification,#_of_cells,genus_size_rank,Domain,Phyla,Class,Order,Family,Genus,Species,Best_classification,Average_gc_content,Average_hypothetical_cds_fraction,median_estimated_diameter,mean_estimated_genome_length,Average_doubling_time,median_doubling_time
0,Unclassified,Unclassified,1790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unclassified,43.870487,0.561344,0.660,6.885372e+06,4.165118,3.478185
1,Pelagibacter,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,628,1.0,Bacteria,Proteobacteria,Alphaproteobacteria,Pelagibacterales,Pelagibacteraceae,Pelagibacter,NaN,Pelagibacter,29.846285,0.220466,0.300,1.464149e+06,8.115193,8.112808
2,SW10,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,326,2.0,Bacteria,Verrucomicrobiota,Verrucomicrobiae,Verrucomicrobiales,Akkermansiaceae,SW10,NaN,SW10,50.074055,0.512254,0.895,4.920788e+06,4.386646,4.062906
3,SCGC-AAA076-P13,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,178,3.0,Bacteria,Proteobacteria,Gammaproteobacteria,SAR86,D2472,SCGC-AAA076-P13,NaN,SCGC-AAA076-P13,34.074901,0.248851,0.315,1.930576e+06,8.312164,8.593175
4,D2472,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,173,4.0,Bacteria,Proteobacteria,Gammaproteobacteria,SAR86,D2472,D2472,NaN,D2472,37.613462,0.239195,0.360,2.020084e+06,9.696105,9.841434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,UBA6669,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,1,292.0,Bacteria,Verrucomicrobiota,Verrucomicrobiae,Opitutales,Opitutaceae,UBA6669,NaN,UBA6669,56.763571,0.430412,0.690,5.651669e+06,1.767563,1.767563
213,UBA721,d__Bacteria;p__Planctomycetota;c__Planctomycet...,1,294.0,Bacteria,Planctomycetota,Planctomycetes,Pirellulales,Pirellulaceae,UBA721,UBA721 sp002705165,UBA721,48.148855,0.462866,0.960,5.444938e+06,9.266202,9.266202
214,UBA7359,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,1,295.0,Bacteria,Proteobacteria,Gammaproteobacteria,UBA4486,UBA4486,UBA7359,NaN,UBA7359,37.587477,0.239183,0.590,2.259463e+06,11.276597,11.276597
215,UBA7446,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,1,296.0,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,UBA7446,NaN,UBA7446,40.534629,0.323387,0.390,2.024038e+06,6.694833,6.694833


In [41]:
master_pivot=master_pivot.merge(GEN_data, left_on='genus', right_on='GTDB_genus', how='left')

master_pivot=master_pivot.drop(['GTDB_genus'], axis=1)
#prokka_pivot['date']=prokka_pivot['date'].astype(str)
master_pivot

,genus,date,(-)-trans-carveol_dehydrogenase:limC:1.1.1.-,(2E%2C6E)-farnesyl_diphosphate_synthase:.:2.5.1.10,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_1:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_2:1.1.1.338,(2R)-sulfolactate_sulfo-lyase_subunit_alpha:suyA:4.4.1.24,(2R)-sulfolactate_sulfo-lyase_subunit_beta:suyB:4.4.1.24,(2S)-methylsuccinyl-CoA_dehydrogenase:mcd:1.3.8.12,...,Family,Genus,Species,Best_classification,Average_gc_content,Average_hypothetical_cds_fraction,median_estimated_diameter,mean_estimated_genome_length,Average_doubling_time,median_doubling_time
0,AAA164-E04,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,AAA164-E04,AAA164-E04,NaN,AAA164-E04,47.554405,0.485247,1.06,5.870149e+06,12.755643,12.456005
1,AAA164-E04,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,AAA164-E04,AAA164-E04,NaN,AAA164-E04,47.554405,0.485247,1.06,5.870149e+06,12.755643,12.456005
2,AAA164-E04,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,AAA164-E04,AAA164-E04,NaN,AAA164-E04,47.554405,0.485247,1.06,5.870149e+06,12.755643,12.456005
3,AAA164-E04,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,AAA164-E04,AAA164-E04,NaN,AAA164-E04,47.554405,0.485247,1.06,5.870149e+06,12.755643,12.456005
4,AAA536-G10,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,AAA536-G10,AAA536-G10,NaN,AAA536-G10,31.655487,0.244558,0.55,2.697551e+06,10.508890,10.541393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,Winogradskyella,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Flavobacteriaceae,Winogradskyella,NaN,Winogradskyella,33.482789,0.415061,0.89,3.568247e+06,4.283599,4.356637
660,Yoonia,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.968799,...,Rhodobacteraceae,Yoonia,NaN,Yoonia,54.596705,0.358001,0.86,4.369587e+06,2.271783,1.606194
661,Yoonia,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Rhodobacteraceae,Yoonia,NaN,Yoonia,54.596705,0.358001,0.86,4.369587e+06,2.271783,1.606194
662,Yoonia,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032036,...,Rhodobacteraceae,Yoonia,NaN,Yoonia,54.596705,0.358001,0.86,4.369587e+06,2.271783,1.606194


In [42]:
biogeo_data=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/GoM_Summary_files/EBB_biogeochemical_measurements.csv')
#biogeo_data['date']=biogeo_data['date'].astype(str)

biogeo_data

,date,SampleTime,SampleLocale,SStemp,AirTemp,SSSalinity,WeatherObs,predHiTide,windDir,windStrength,...,ChlWWug/L,PhaeoWWug/L,Comments,Bulk O2 consumption (uM),Total cells/ml,RSG+ cells per ml,NO3+NO2 (um),NO2 (um),NH4 (um),PO4 (um)
0,20170412,8:40,ebb,5.8,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"It was a warm, sunny morning. Copepods were vi...",6.50,1.88E+06,1350000.0,NaN,NaN,NaN,NaN
1,20170818,9:20,ebb,15.6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.11,1.42E+06,790000.0,NaN,NaN,NaN,NaN
2,20171102,9:05,ebb,12.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20181030,8:20,ebb,9.5,5.7,34.0,rain previous day,15:50,NaN,NaN,...,1.74,NaN,special G2P collection- not at high tide,NaN,NaN,NaN,5.61,0.28,1.05,0.81
4,20190402,8:15,ebb,3.2,3.8,32.0,NaN,10:19,NaN,1.0,...,2.91,0.62,G2P collection,NaN,NaN,NaN,1.53,0.03,0.55,0.36
5,20190709,8:00,ebb,12.7,17.2,31.0,NaN,5:19,NaN,1.0,...,3.06,0.97,G2P collection,NaN,NaN,NaN,0.59,0.04,1.79,0.31


In [43]:

master_pivot=master_pivot.merge(biogeo_data, on='date', how='left')
master_pivot.to_csv(outdir+ 'Prokka_Transcripts_percell_O2_percell_Genus_stats_biogeochemical_master.csv')

master_pivot

,genus,date,(-)-trans-carveol_dehydrogenase:limC:1.1.1.-,(2E%2C6E)-farnesyl_diphosphate_synthase:.:2.5.1.10,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_1:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_2:1.1.1.338,(2R)-sulfolactate_sulfo-lyase_subunit_alpha:suyA:4.4.1.24,(2R)-sulfolactate_sulfo-lyase_subunit_beta:suyB:4.4.1.24,(2S)-methylsuccinyl-CoA_dehydrogenase:mcd:1.3.8.12,...,ChlWWug/L,PhaeoWWug/L,Comments,Bulk O2 consumption (uM),Total cells/ml,RSG+ cells per ml,NO3+NO2 (um),NO2 (um),NH4 (um),PO4 (um)
0,AAA164-E04,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAA164-E04,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.74,NaN,special G2P collection- not at high tide,NaN,NaN,NaN,5.61,0.28,1.05,0.81
2,AAA164-E04,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.91,0.62,G2P collection,NaN,NaN,NaN,1.53,0.03,0.55,0.36
3,AAA164-E04,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.06,0.97,G2P collection,NaN,NaN,NaN,0.59,0.04,1.79,0.31
4,AAA536-G10,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,Winogradskyella,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.06,0.97,G2P collection,NaN,NaN,NaN,0.59,0.04,1.79,0.31
660,Yoonia,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.968799,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
661,Yoonia,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.74,NaN,special G2P collection- not at high tide,NaN,NaN,NaN,5.61,0.28,1.05,0.81
662,Yoonia,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032036,...,2.91,0.62,G2P collection,NaN,NaN,NaN,1.53,0.03,0.55,0.36


In [44]:
master_data=pd.read_csv(outdir+ 'Prokka_Transcripts_percell_O2_percell_Genus_stats_biogeochemical_master.csv',
                       index_col=0)

master_data=master_data[master_data.date != 20171102]
master_data.to_csv(outdir+ 'Prokka_Transcripts_percell_O2_percell_Genus_stats_biogeochemical_master_no2017.csv')
master_data

/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (13622) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,genus,date,(-)-trans-carveol_dehydrogenase:limC:1.1.1.-,(2E%2C6E)-farnesyl_diphosphate_synthase:.:2.5.1.10,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_1:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_2:1.1.1.338,(2R)-sulfolactate_sulfo-lyase_subunit_alpha:suyA:4.4.1.24,(2R)-sulfolactate_sulfo-lyase_subunit_beta:suyB:4.4.1.24,(2S)-methylsuccinyl-CoA_dehydrogenase:mcd:1.3.8.12,...,ChlWWug/L,PhaeoWWug/L,Comments,Bulk O2 consumption (uM),Total cells/ml,RSG+ cells per ml,NO3+NO2 (um),NO2 (um),NH4 (um),PO4 (um)
1,AAA164-E04,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.74,NaN,special G2P collection- not at high tide,NaN,NaN,NaN,5.61,0.28,1.05,0.81
2,AAA164-E04,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.91,0.62,G2P collection,NaN,NaN,NaN,1.53,0.03,0.55,0.36
3,AAA164-E04,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.06,0.97,G2P collection,NaN,NaN,NaN,0.59,0.04,1.79,0.31
5,AAA536-G10,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.74,NaN,special G2P collection- not at high tide,NaN,NaN,NaN,5.61,0.28,1.05,0.81
6,AAA536-G10,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007194,...,2.91,0.62,G2P collection,NaN,NaN,NaN,1.53,0.03,0.55,0.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,Winogradskyella,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.91,0.62,G2P collection,NaN,NaN,NaN,1.53,0.03,0.55,0.36
659,Winogradskyella,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.06,0.97,G2P collection,NaN,NaN,NaN,0.59,0.04,1.79,0.31
661,Yoonia,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.74,NaN,special G2P collection- not at high tide,NaN,NaN,NaN,5.61,0.28,1.05,0.81
662,Yoonia,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032036,...,2.91,0.62,G2P collection,NaN,NaN,NaN,1.53,0.03,0.55,0.36


In [45]:
with open("/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/Genera_with_10+_SAGs.txt") as file:
    tenGENs_list = [line.strip() for line in file]


tenSAG_master=master_data[master_data['genus'].isin(tenGENs_list)]

tenSAG_master.to_csv(outdir+ 'Prokka_Transcripts_percell_O2_percell_10SAG_genera_stats_biogeochemical_master_no2017.csv')


tenSAG_master

,genus,date,(-)-trans-carveol_dehydrogenase:limC:1.1.1.-,(2E%2C6E)-farnesyl_diphosphate_synthase:.:2.5.1.10,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_1:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_2:1.1.1.338,(2R)-sulfolactate_sulfo-lyase_subunit_alpha:suyA:4.4.1.24,(2R)-sulfolactate_sulfo-lyase_subunit_beta:suyB:4.4.1.24,(2S)-methylsuccinyl-CoA_dehydrogenase:mcd:1.3.8.12,...,ChlWWug/L,PhaeoWWug/L,Comments,Bulk O2 consumption (uM),Total cells/ml,RSG+ cells per ml,NO3+NO2 (um),NO2 (um),NH4 (um),PO4 (um)
1,AAA164-E04,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.74,NaN,special G2P collection- not at high tide,NaN,NaN,NaN,5.61,0.28,1.05,0.81
2,AAA164-E04,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.91,0.62,G2P collection,NaN,NaN,NaN,1.53,0.03,0.55,0.36
3,AAA164-E04,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.06,0.97,G2P collection,NaN,NaN,NaN,0.59,0.04,1.79,0.31
5,AAA536-G10,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.74,NaN,special G2P collection- not at high tide,NaN,NaN,NaN,5.61,0.28,1.05,0.81
6,AAA536-G10,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007194,...,2.91,0.62,G2P collection,NaN,NaN,NaN,1.53,0.03,0.55,0.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,UBA9926,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.141807,...,2.91,0.62,G2P collection,NaN,NaN,NaN,1.53,0.03,0.55,0.36
635,UBA9926,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.011971,...,3.06,0.97,G2P collection,NaN,NaN,NaN,0.59,0.04,1.79,0.31
641,Unclassified,20181030,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.000000,...,1.74,NaN,special G2P collection- not at high tide,NaN,NaN,NaN,5.61,0.28,1.05,0.81
642,Unclassified,20190402,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.000000,...,2.91,0.62,G2P collection,NaN,NaN,NaN,1.53,0.03,0.55,0.36


# Normalize the transcripts and remerge

In [46]:
import scipy.stats as stats
prokka_norm=prokka_pivot.reset_index()
prokka_norm.set_index(['genus', 'date'],inplace=True)
prokka_norm

prokka_annotation         (-)-trans-carveol_dehydrogenase:limC:1.1.1.-  \
genus           date                                                     
AAA164-E04      20171102                                           NaN   
                20181030                                           NaN   
                20190402                                           NaN   
                20190709                                           NaN   
AAA536-G10      20171102                                           NaN   
...                                                                ...   
Winogradskyella 20190709                                           NaN   
Yoonia          20171102                                           NaN   
                20181030                                           NaN   
                20190402                                           NaN   
                20190709                                           NaN   

prokka_annotation         (2E%2C6E)-farnesyl_diphosphate_synthase:.:2.5.1.10  \
genus           date                                                           
AAA164-E04      20171102                                                NaN    
                20181030                                                NaN    
                20190402                                                NaN    
                20190709                                                NaN    
AAA536-G10      20171102                                                NaN    
...                                                                     ...    
Winogradskyella 20190709                                                NaN    
Yoonia          20171102                                                NaN    
                20181030                                                NaN    
                20190402                                                NaN    
                20190709                                                NaN    

prokka_annotation         (2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC:1.1.1.338  \
genus           date                                                                   
AAA164-E04      20171102                                                NaN            
                20181030                                                NaN            
                20190402                                                NaN            
                20190709                                                NaN            
AAA536-G10      20171102                                                NaN            
...                                                                     ...            
Winogradskyella 20190709                                                NaN            
Yoonia          20171102                                                NaN            
                20181030                                                NaN            
                20190402                                                NaN            
                20190709                                                NaN            

prokka_annotation         (2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_1:1.1.1.338  \
genus           date                                                                     
AAA164-E04      20171102                                                NaN              
                20181030                                                NaN              
                20190402                                                NaN              
                20190709                                                NaN              
AAA536-G10      20171102                                                NaN              
...                                                                     ...              
Winogradskyella 20190709                                                NaN              
Yoonia          20171102                 

In [47]:
# DO THE Zscore normalization

prokka_norm=prokka_norm.T

znorm=(prokka_norm - prokka_norm.mean())/prokka_norm.std()
znorm.to_csv(outdir+'normalization_test.csv')

znorm=znorm.T
znorm


prokka_annotation         (-)-trans-carveol_dehydrogenase:limC:1.1.1.-  \
genus           date                                                     
AAA164-E04      20171102                                           NaN   
                20181030                                           NaN   
                20190402                                           NaN   
                20190709                                           NaN   
AAA536-G10      20171102                                           NaN   
...                                                                ...   
Winogradskyella 20190709                                           NaN   
Yoonia          20171102                                           NaN   
                20181030                                           NaN   
                20190402                                           NaN   
                20190709                                           NaN   

prokka_annotation         (2E%2C6E)-farnesyl_diphosphate_synthase:.:2.5.1.10  \
genus           date                                                           
AAA164-E04      20171102                                                NaN    
                20181030                                                NaN    
                20190402                                                NaN    
                20190709                                                NaN    
AAA536-G10      20171102                                                NaN    
...                                                                     ...    
Winogradskyella 20190709                                                NaN    
Yoonia          20171102                                                NaN    
                20181030                                                NaN    
                20190402                                                NaN    
                20190709                                                NaN    

prokka_annotation         (2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC:1.1.1.338  \
genus           date                                                                   
AAA164-E04      20171102                                                NaN            
                20181030                                                NaN            
                20190402                                                NaN            
                20190709                                                NaN            
AAA536-G10      20171102                                                NaN            
...                                                                     ...            
Winogradskyella 20190709                                                NaN            
Yoonia          20171102                                                NaN            
                20181030                                                NaN            
                20190402                                                NaN            
                20190709                                                NaN            

prokka_annotation         (2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_1:1.1.1.338  \
genus           date                                                                     
AAA164-E04      20171102                                                NaN              
                20181030                                                NaN              
                20190402                                                NaN              
                20190709                                                NaN              
AAA536-G10      20171102                                                NaN              
...                                                                     ...              
Winogradskyella 20190709                                                NaN              
Yoonia          20171102                 

In [48]:
znorm=znorm.merge(O2_data, on=['genus', 'date'])

znorm

,genus,date,(-)-trans-carveol_dehydrogenase:limC:1.1.1.-,(2E%2C6E)-farnesyl_diphosphate_synthase:.:2.5.1.10,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_1:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_2:1.1.1.338,(2R)-sulfolactate_sulfo-lyase_subunit_alpha:suyA:4.4.1.24,(2R)-sulfolactate_sulfo-lyase_subunit_beta:suyB:4.4.1.24,(2S)-methylsuccinyl-CoA_dehydrogenase:mcd:1.3.8.12,...,tRNA_threonylcarbamoyladenosine_biosynthesis_protein_TsaE:tsaE_1:.,tRNA_threonylcarbamoyladenosine_dehydratase:tcdA:6.1.-.-,tRNA_uridine_5-carboxymethylaminomethyl_modification_enzyme_MnmG:mnmG:.,tRNA_uridine_5-carboxymethylaminomethyl_modification_enzyme_MnmG:mnmG_1:.,tRNA_uridine_5-carboxymethylaminomethyl_modification_enzyme_MnmG:mnmG_2:.,tRNA_uridine_5-carboxymethylaminomethyl_modification_enzyme_MnmG:mnmG_3:.,zeta-carotene-forming_phytoene_desaturase:carA2:1.3.99.29,zeta-carotene-forming_phytoene_desaturase:carA2_1:1.3.99.29,zeta-carotene-forming_phytoene_desaturase:carA2_2:1.3.99.29,Weighted_avg_O2_consumed_per_cell
0,AAA164-E04,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.044946,NaN,NaN,NaN,NaN,NaN,NaN,0.002330
1,AAA164-E04,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002050
2,AAA164-E04,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002101
3,AAA164-E04,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.009905,NaN,NaN,NaN,NaN,NaN,NaN,0.001999
4,AAA536-G10,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.029921,NaN,NaN,NaN,NaN,NaN,NaN,0.002330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,Winogradskyella,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.083635,NaN,0.056376,NaN,NaN,NaN,NaN,NaN,NaN,0.001999
656,Yoonia,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.223528,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002330
657,Yoonia,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002050
658,Yoonia,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.072962,...,NaN,NaN,-0.095966,NaN,NaN,NaN,NaN,NaN,NaN,0.002101


In [49]:
znorm=znorm.merge(GEN_data, left_on='genus', right_on='GTDB_genus', how='left')

znorm=znorm.drop(['GTDB_genus'], axis=1)
#prokka_pivot['date']=prokka_pivot['date'].astype(str)
znorm

,genus,date,(-)-trans-carveol_dehydrogenase:limC:1.1.1.-,(2E%2C6E)-farnesyl_diphosphate_synthase:.:2.5.1.10,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_1:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_2:1.1.1.338,(2R)-sulfolactate_sulfo-lyase_subunit_alpha:suyA:4.4.1.24,(2R)-sulfolactate_sulfo-lyase_subunit_beta:suyB:4.4.1.24,(2S)-methylsuccinyl-CoA_dehydrogenase:mcd:1.3.8.12,...,Family,Genus,Species,Best_classification,Average_gc_content,Average_hypothetical_cds_fraction,median_estimated_diameter,mean_estimated_genome_length,Average_doubling_time,median_doubling_time
0,AAA164-E04,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,AAA164-E04,AAA164-E04,NaN,AAA164-E04,47.554405,0.485247,1.06,5.870149e+06,12.755643,12.456005
1,AAA164-E04,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,AAA164-E04,AAA164-E04,NaN,AAA164-E04,47.554405,0.485247,1.06,5.870149e+06,12.755643,12.456005
2,AAA164-E04,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,AAA164-E04,AAA164-E04,NaN,AAA164-E04,47.554405,0.485247,1.06,5.870149e+06,12.755643,12.456005
3,AAA164-E04,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,AAA164-E04,AAA164-E04,NaN,AAA164-E04,47.554405,0.485247,1.06,5.870149e+06,12.755643,12.456005
4,AAA536-G10,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,AAA536-G10,AAA536-G10,NaN,AAA536-G10,31.655487,0.244558,0.55,2.697551e+06,10.508890,10.541393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,Winogradskyella,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Flavobacteriaceae,Winogradskyella,NaN,Winogradskyella,33.482789,0.415061,0.89,3.568247e+06,4.283599,4.356637
660,Yoonia,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.223528,...,Rhodobacteraceae,Yoonia,NaN,Yoonia,54.596705,0.358001,0.86,4.369587e+06,2.271783,1.606194
661,Yoonia,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Rhodobacteraceae,Yoonia,NaN,Yoonia,54.596705,0.358001,0.86,4.369587e+06,2.271783,1.606194
662,Yoonia,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.072962,...,Rhodobacteraceae,Yoonia,NaN,Yoonia,54.596705,0.358001,0.86,4.369587e+06,2.271783,1.606194


In [50]:

znorm=znorm.merge(biogeo_data, on='date', how='left')
znorm.to_csv(outdir+ 'Prokka_znormalized_Transcripts_percell_O2_percell_Genus_stats_biogeochemical_master.csv')

znorm

,genus,date,(-)-trans-carveol_dehydrogenase:limC:1.1.1.-,(2E%2C6E)-farnesyl_diphosphate_synthase:.:2.5.1.10,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_1:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_2:1.1.1.338,(2R)-sulfolactate_sulfo-lyase_subunit_alpha:suyA:4.4.1.24,(2R)-sulfolactate_sulfo-lyase_subunit_beta:suyB:4.4.1.24,(2S)-methylsuccinyl-CoA_dehydrogenase:mcd:1.3.8.12,...,ChlWWug/L,PhaeoWWug/L,Comments,Bulk O2 consumption (uM),Total cells/ml,RSG+ cells per ml,NO3+NO2 (um),NO2 (um),NH4 (um),PO4 (um)
0,AAA164-E04,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAA164-E04,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.74,NaN,special G2P collection- not at high tide,NaN,NaN,NaN,5.61,0.28,1.05,0.81
2,AAA164-E04,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.91,0.62,G2P collection,NaN,NaN,NaN,1.53,0.03,0.55,0.36
3,AAA164-E04,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.06,0.97,G2P collection,NaN,NaN,NaN,0.59,0.04,1.79,0.31
4,AAA536-G10,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,Winogradskyella,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.06,0.97,G2P collection,NaN,NaN,NaN,0.59,0.04,1.79,0.31
660,Yoonia,20171102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.223528,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
661,Yoonia,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.74,NaN,special G2P collection- not at high tide,NaN,NaN,NaN,5.61,0.28,1.05,0.81
662,Yoonia,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.072962,...,2.91,0.62,G2P collection,NaN,NaN,NaN,1.53,0.03,0.55,0.36


In [51]:
znorm=pd.read_csv(outdir+ 'Prokka_znormalized_Transcripts_percell_O2_percell_Genus_stats_biogeochemical_master.csv',
                       index_col=0)

znorm=znorm[znorm.date != 20171102]
znorm.to_csv(outdir+ 'Prokka_znormalized_Transcripts_percell_O2_percell_Genus_stats_biogeochemical_master_no2017.csv')


/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (13622) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [52]:
with open("/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/Genera_with_10+_SAGs.txt") as file:
    tenGENs_list = [line.strip() for line in file]


tenSAG_master=znorm[znorm['genus'].isin(tenGENs_list)]

tenSAG_master.to_csv(outdir+ 'Prokka_znormalized_Transcripts_percell_O2_percell_10SAG_genera_stats_biogeochemical_master_no2017.csv')

tenSAG_master

,genus,date,(-)-trans-carveol_dehydrogenase:limC:1.1.1.-,(2E%2C6E)-farnesyl_diphosphate_synthase:.:2.5.1.10,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_1:1.1.1.338,(2R)-3-sulfolactate_dehydrogenase_(NADP(+)):comC_2:1.1.1.338,(2R)-sulfolactate_sulfo-lyase_subunit_alpha:suyA:4.4.1.24,(2R)-sulfolactate_sulfo-lyase_subunit_beta:suyB:4.4.1.24,(2S)-methylsuccinyl-CoA_dehydrogenase:mcd:1.3.8.12,...,ChlWWug/L,PhaeoWWug/L,Comments,Bulk O2 consumption (uM),Total cells/ml,RSG+ cells per ml,NO3+NO2 (um),NO2 (um),NH4 (um),PO4 (um)
1,AAA164-E04,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.74,NaN,special G2P collection- not at high tide,NaN,NaN,NaN,5.61,0.28,1.05,0.81
2,AAA164-E04,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.91,0.62,G2P collection,NaN,NaN,NaN,1.53,0.03,0.55,0.36
3,AAA164-E04,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.06,0.97,G2P collection,NaN,NaN,NaN,0.59,0.04,1.79,0.31
5,AAA536-G10,20181030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.74,NaN,special G2P collection- not at high tide,NaN,NaN,NaN,5.61,0.28,1.05,0.81
6,AAA536-G10,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.108396,...,2.91,0.62,G2P collection,NaN,NaN,NaN,1.53,0.03,0.55,0.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,UBA9926,20190402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.076733,...,2.91,0.62,G2P collection,NaN,NaN,NaN,1.53,0.03,0.55,0.36
635,UBA9926,20190709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.113828,...,3.06,0.97,G2P collection,NaN,NaN,NaN,0.59,0.04,1.79,0.31
641,Unclassified,20181030,NaN,-0.015621,-0.015621,NaN,NaN,-0.015621,-0.015621,-0.015621,...,1.74,NaN,special G2P collection- not at high tide,NaN,NaN,NaN,5.61,0.28,1.05,0.81
642,Unclassified,20190402,NaN,-0.014331,-0.014331,NaN,NaN,-0.014331,-0.014331,-0.014331,...,2.91,0.62,G2P collection,NaN,NaN,NaN,1.53,0.03,0.55,0.36
